Librerías del Proyecto

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from nltk.tokenize import word_tokenize
import os
 
import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')
 
# Importing libraries necessary for Model Building and Training
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
 
import warnings
warnings.filterwarnings('ignore')

import mailbox
from email import policy
from email.parser import BytesParser

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
archivo_correo = mailbox.mbox(os.getcwd() + r'\easy_ham\0001.ea7e79d3153e7469e7a9c3e0af6a357e')

with open(os.getcwd() + r'\easy_ham\0001.ea7e79d3153e7469e7a9c3e0af6a357e', 'rb') as fp:
    archivo_correo_2 = BytesParser(policy=policy.default).parse(fp)

archivo_correo_2.get_payload()


'    Date:        Wed, 21 Aug 2002 10:54:46 -0500\n    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>\n    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>\n\n\n  | I can\'t reproduce this error.\n\nFor me it is very repeatable... (like every time, without fail).\n\nThis is the debug log of the pick happening ...\n\n18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}\n18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury\n18:19:04 Ftoc_PickMsgs {{1 hit}}\n18:19:04 Marking 1 hits\n18:19:04 tkerror: syntax error in expression "int ...\n\nNote, if I run the pick command by hand ...\n\ndelta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury\n1 hit\n\nThat\'s where the "1 hit" comes from (obviously).  The version of nmh I\'m\nusing is ...\n\ndelta$ pick -version\npick -- nmh-1.0.4 [compiled on fuchsia.c

In [67]:

for mensaje in archivo_correo:
    print(mensaje.is_multipart())
    cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
cuerpo

False


'    Date:        Wed, 21 Aug 2002 10:54:46 -0500\n    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>\n    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>\n\n\n  | I can\'t reproduce this error.\n\nFor me it is very repeatable... (like every time, without fail).\n\nThis is the debug log of the pick happening ...\n\n18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}\n18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury\n18:19:04 Ftoc_PickMsgs {{1 hit}}\n18:19:04 Marking 1 hits\n18:19:04 tkerror: syntax error in expression "int ...\n\nNote, if I run the pick command by hand ...\n\ndelta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury\n1 hit\n\nThat\'s where the "1 hit" comes from (obviously).  The version of nmh I\'m\nusing is ...\n\ndelta$ pick -version\npick -- nmh-1.0.4 [compiled on fuchsia.c

Abriendo las carpetas de spam y no spam:

In [68]:
os.getcwd()

'c:\\Users\\danie\\source\\repos\\I7041\\proyecto'

In [69]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
            if c == '<' and not quote:
                tag = True
            elif c == '>' and not quote:
                tag = False
            elif (c == '"' or c == "'") and tag:
                quote = not quote
            elif not tag:
                out = out + c

    return out

In [77]:
def email_parser(mbox, carp):
    message = {}
    for mensaje in mbox:
        cuerpo = ""
        if mensaje.is_multipart():
            # Si el correo tiene varias partes, buscar la parte de texto/html
            message['isMultipart'] = True
            for parte in mensaje.get_payload():
                if parte.get_content_type() == 'text/plain':
                    cuerpo = parte.get_payload(decode=True).decode(errors='replace')
                    #cuerpo += cuerpo.translate(str.maketrans('', '', string.punctuation))
                if parte.get_content_type() == 'text/html':
                    cuerpo = parte.get_payload(decode=True).decode(errors='replace')
                    #cuerpo += remove_html_markup(cuerpo)
        else:
            message['isMultipart'] = False
            # Si el correo es de una sola parte, obtener el cuerpo directamente
            if mensaje.get_content_type() == 'text/plain':
                char_set = mensaje.get_charset()
                cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
                #cuerpo = cuerpo.translate(str.maketrans('', '', string.punctuation))
            if mensaje.get_content_type() == 'text/html':
                char_set = mensaje.get_charset()
                cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
                #cuerpo = remove_html_markup(cuerpo)
        message['Text'] = cuerpo
        message['isSpam'] = carp

        return message

In [78]:
carpeta = ["spam", "easy_ham"]  # Ruta de la carpeta que deseas leer
message_list = []
for carp in carpeta:
    # Obtener la lista de archivos en la carpeta
    lista_archivos = [f'{os.path.join(os.getcwd(), carp)}\\{x}' for x in os.listdir(os.path.join(os.getcwd(), carp))]
    #print(lista_archivos)
    # Leer cada archivo de la lista
    for archivo in lista_archivos:
        if os.path.isfile(archivo):  # Verificar que sea un archivo
            # Abrir el dataset de correos
            mbox = mailbox.mbox(archivo)
            # Recorrer cada correo en el dataset
            message = email_parser(mbox, carp)
            if message == None:
                print(archivo)
            else:
                
                message_list.append(message)

c:\Users\danie\source\repos\I7041\proyecto\spam\0000.7b1b73cf36cf9dbc3d64e3f2ee2b91f1
c:\Users\danie\source\repos\I7041\proyecto\spam\0035.8e582263070076dfe6000411d9b13ce6
c:\Users\danie\source\repos\I7041\proyecto\spam\0036.8e582263070076dfe6000411d9b13ce6
c:\Users\danie\source\repos\I7041\proyecto\spam\0037.7ce3307b56dd90453027a6630179282e
c:\Users\danie\source\repos\I7041\proyecto\spam\0038.7ce3307b56dd90453027a6630179282e
c:\Users\danie\source\repos\I7041\proyecto\spam\0039.256602e2cb5a5b373bdd1fb631d9f452
c:\Users\danie\source\repos\I7041\proyecto\spam\0040.256602e2cb5a5b373bdd1fb631d9f452
c:\Users\danie\source\repos\I7041\proyecto\spam\0041.21cc985cc36d931916863aed24de8c27
c:\Users\danie\source\repos\I7041\proyecto\spam\0042.21cc985cc36d931916863aed24de8c27
c:\Users\danie\source\repos\I7041\proyecto\spam\0043.8d93819b95ff90bf2e2b141c2909bfc9
c:\Users\danie\source\repos\I7041\proyecto\spam\0044.889d785885f092c269741b11f2124dce
c:\Users\danie\source\repos\I7041\proyecto\spam\0266.9

In [79]:
message_list

[{'isMultipart': False,
  'Text': '<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">\n<HTML><HEAD>\n<META content="text/html; charset=windows-1252" http-equiv=Content-Type>\n<META content="MSHTML 5.00.2314.1000" name=GENERATOR></HEAD>\n<BODY><!-- Inserted by Calypso -->\n<TABLE border=0 cellPadding=0 cellSpacing=2 id=_CalyPrintHeader_ rules=none \nstyle="COLOR: black; DISPLAY: none" width="100%">\n  <TBODY>\n  <TR>\n    <TD colSpan=3>\n      <HR color=black noShade SIZE=1>\n    </TD></TR></TD></TR>\n  <TR>\n    <TD colSpan=3>\n      <HR color=black noShade SIZE=1>\n    </TD></TR></TBODY></TABLE><!-- End Calypso --><!-- Inserted by Calypso --><FONT \ncolor=#000000 face=VERDANA,ARIAL,HELVETICA size=-2><BR></FONT></TD></TR></TABLE><!-- End Calypso --><FONT color=#ff0000 \nface="Copperplate Gothic Bold" size=5 PTSIZE="10">\n<CENTER>Save up to 70% on Life Insurance.</CENTER></FONT><FONT color=#ff0000 \nface="Copperplate Gothic Bold" size=5 PTSIZE="10">\n<CENTER>Why Spend More T